## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
from config import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../vacation_search/weatherpy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Hilo,US,80.60,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
1,7,Harper,LR,78.19,broken clouds,4.3750,-7.7169,Screensaver
2,10,Port Blair,IN,81.52,broken clouds,11.6667,92.7500,Welcomhotel Bay Island Port Blair
3,11,Mitsamiouli,KM,81.97,broken clouds,-11.3847,43.2844,Foyer ADM
4,12,Quatre Cocos,MU,77.00,scattered clouds,-20.2078,57.7625,Emeraude Beach Attitude Hotel


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

In [4]:
# 4a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[(vacation_df["Hotel Name"] == "Hotel Tesoro Los Cabos") & (vacation_df["City"] == "Cabo San Lucas")]
vacation_end = vacation_df.loc[(vacation_df["Hotel Name"] == "Hotel Tesoro Los Cabos") & (vacation_df["City"] == "Cabo San Lucas")]
vacation_stop1 = vacation_df.loc[(vacation_df["Hotel Name"] == "HS Hotsson Smart Acapulco") & (vacation_df["City"] == "Acapulco")]
vacation_stop2 = vacation_df.loc[(vacation_df["Hotel Name"] == "Hotel Posada San Jose") & (vacation_df["City"] == "Pochutla")]
vacation_stop3 = vacation_df.loc[(vacation_df["Hotel Name"] == "Trump International Hotel & Tower Panama") & (vacation_df["City"] == "Panama City")]

vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
222,519,Cabo San Lucas,MX,68.0,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos


In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = (pd.DataFrame(vacation_start).to_numpy()[0][5],pd.DataFrame(vacation_start).to_numpy()[0][6])
end = (pd.DataFrame(vacation_end).to_numpy()[0][5],pd.DataFrame(vacation_end).to_numpy()[0][6])
stop1 = (pd.DataFrame(vacation_stop1).to_numpy()[0][5],pd.DataFrame(vacation_stop1).to_numpy()[0][6])
stop2 = (pd.DataFrame(vacation_stop2).to_numpy()[0][5],pd.DataFrame(vacation_stop2).to_numpy()[0][6])
stop3 = (pd.DataFrame(vacation_stop3).to_numpy()[0][5],pd.DataFrame(vacation_stop3).to_numpy()[0][6])

start

(22.8909, -109.9124)

In [7]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()

start2end_via_stop1_stop2_stop3 = gmaps.directions_layer(start, end, waypoints = [stop1, stop2, stop3], show_markers=True,
    stroke_color='blue', stroke_weight=3.0, stroke_opacity=1.0, travel_mode='DRIVING')
fig.add_layer(start2end_via_stop1_stop2_stop3)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, 
                          vacation_stop2, vacation_stop3]
                         ,ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,519,Cabo San Lucas,MX,68.0,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
1,296,Acapulco,MX,75.2,clear sky,16.8634,-99.8901,HS Hotsson Smart Acapulco
2,498,Pochutla,MX,77.0,clear sky,15.7432,-96.4661,Hotel Posada San Jose
3,638,Panama City,PA,80.6,broken clouds,8.9936,-79.5197,Trump International Hotel & Tower Panama


In [9]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

In [10]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))